## 데이터 준비

In [1]:
import pandas as pd
df = pd.read_csv('imdb.zip')

In [2]:
df.head()

,review,sentiment
0,"A very, very, very slow-moving, aimless movie ...",0
1,Not sure who was more lost - the flat characte...,0
2,Attempting artiness with black & white and cle...,0
3,Very little music or anything to speak of.,0
4,The best scene in the movie was when Gerardo i...,1


In [3]:
import joblib
tk = joblib.load('tokenizer.pkl')

In [4]:
from sklearn.model_selection import train_test_split
review_train, review_test, y_train, y_test = train_test_split(df['review'],df['sentiment'],test_size=0.2,random_state=42)

In [5]:
seqs = tk.texts_to_sequences(review_train)

In [6]:
seqs[0]

[9, 6, 33, 1258, 214]

In [7]:
seqs[:3]

[[9, 6, 33, 1258, 214],
 [178, 5, 28, 35, 23, 168, 713, 591, 3, 713, 1, 10, 1, 280],
 [206, 336, 4]]

## 순방향 순환신경망

In [8]:
import tensorflow as tf

In [9]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs,maxlen=None,padding='pre',truncating='pre')

In [14]:
NUM_WORDS = tk.num_words + 1

In [15]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8),
    tf.keras.layers.Dense(1,activation='sigmoid')

])

In [16]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 8)           16008     
                                                                 
 lstm (LSTM)                 (None, 8)                 544       
                                                                 
 dense (Dense)               (None, 1)                 9         
                                                                 
Total params: 16,561
Trainable params: 16,561
Non-trainable params: 0
_________________________________________________________________


In [17]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
model.fit(pads,y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 3s 14ms/step - loss: 0.6927 - accuracy: 0.5288
Epoch 2/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6893 - accuracy: 0.6212
Epoch 3/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6818 - accuracy: 0.7663
Epoch 4/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6579 - accuracy: 0.8150
Epoch 5/10
25/25 [==============================] - 0s 13ms/step - loss: 0.5849 - accuracy: 0.8225
Epoch 6/10
25/25 [==============================] - 0s 13ms/step - loss: 0.4761 - accuracy: 0.8462
Epoch 7/10
25/25 [==============================] - 0s 13ms/step - loss: 0.3863 - accuracy: 0.8863
Epoch 8/10
25/25 [==============================] - 0s 13ms/step - loss: 0.3243 - accuracy: 0.9162
Epoch 9/10
25/25 [==============================] - 0s 13ms/step - loss: 0.2823 - accuracy: 0.9237
Epoch 10/10
25/25 [==============================] - 0s 14ms/step - loss: 0.2403 - accuracy: 0.9500


## 역방향 순환신경망

### pre padding 적용

In [22]:
# 패딩을 post로 처리하는 것이 좋음, 순방향일 경우는 pre로 처리하는것을 권장
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),
    tf.keras.layers.Dense(1,activation='sigmoid')

])

In [23]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [24]:
model.fit(pads,y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 14ms/step - loss: 0.6930 - accuracy: 0.5075
Epoch 2/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6888 - accuracy: 0.6350
Epoch 3/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6794 - accuracy: 0.7550
Epoch 4/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6429 - accuracy: 0.7937
Epoch 5/10
25/25 [==============================] - 0s 14ms/step - loss: 0.5496 - accuracy: 0.8475
Epoch 6/10
25/25 [==============================] - 0s 17ms/step - loss: 0.4398 - accuracy: 0.8950
Epoch 7/10
25/25 [==============================] - 0s 14ms/step - loss: 0.3606 - accuracy: 0.9312
Epoch 8/10
25/25 [==============================] - 0s 15ms/step - loss: 0.3041 - accuracy: 0.9475
Epoch 9/10
25/25 [==============================] - 0s 13ms/step - loss: 0.2681 - accuracy: 0.9475
Epoch 10/10
25/25 [==============================] - 0s 13ms/step - loss: 0.2469 - accuracy: 0.9450


### post padding 적용

In [25]:
pads = tf.keras.preprocessing.sequence.pad_sequences(seqs,maxlen=None,padding='post',truncating='pre')

In [26]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.LSTM(8, go_backwards=True),
    tf.keras.layers.Dense(1,activation='sigmoid')

])

In [27]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [28]:
model.fit(pads,y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 2s 13ms/step - loss: 0.6929 - accuracy: 0.5225
Epoch 2/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6910 - accuracy: 0.6037
Epoch 3/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6870 - accuracy: 0.6525
Epoch 4/10
25/25 [==============================] - 0s 14ms/step - loss: 0.6754 - accuracy: 0.7812
Epoch 5/10
25/25 [==============================] - 0s 13ms/step - loss: 0.6401 - accuracy: 0.7525
Epoch 6/10
25/25 [==============================] - 0s 13ms/step - loss: 0.5593 - accuracy: 0.8150
Epoch 7/10
25/25 [==============================] - 0s 13ms/step - loss: 0.4408 - accuracy: 0.9050
Epoch 8/10
25/25 [==============================] - 0s 13ms/step - loss: 0.3387 - accuracy: 0.9438
Epoch 9/10
25/25 [==============================] - 0s 13ms/step - loss: 0.2640 - accuracy: 0.9575
Epoch 10/10
25/25 [==============================] - 0s 14ms/step - loss: 0.2107 - accuracy: 0.9625


## 양방향 순환신경망

In [31]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(NUM_WORDS, 8, mask_zero=True),
    tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(8)),
    tf.keras.layers.Dense(1,activation='sigmoid')

])

In [32]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_5 (Embedding)     (None, None, 8)           16008     
                                                                 
 bidirectional (Bidirectiona  (None, 16)               1088      
 l)                                                              
                                                                 
 dense_3 (Dense)             (None, 1)                 17        
                                                                 
Total params: 17,113
Trainable params: 17,113
Non-trainable params: 0
_________________________________________________________________


In [33]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [34]:
model.fit(pads,y_train.values, epochs=10)

Epoch 1/10
25/25 [==============================] - 4s 16ms/step - loss: 0.6920 - accuracy: 0.5638
Epoch 2/10
25/25 [==============================] - 0s 16ms/step - loss: 0.6849 - accuracy: 0.7837
Epoch 3/10
25/25 [==============================] - 0s 16ms/step - loss: 0.6665 - accuracy: 0.8388
Epoch 4/10
25/25 [==============================] - 0s 16ms/step - loss: 0.6066 - accuracy: 0.8450
Epoch 5/10
25/25 [==============================] - 0s 16ms/step - loss: 0.4500 - accuracy: 0.8938
Epoch 6/10
25/25 [==============================] - 0s 16ms/step - loss: 0.2918 - accuracy: 0.9388
Epoch 7/10
25/25 [==============================] - 0s 16ms/step - loss: 0.2040 - accuracy: 0.9538
Epoch 8/10
25/25 [==============================] - 0s 16ms/step - loss: 0.1611 - accuracy: 0.9575
Epoch 9/10
25/25 [==============================] - 0s 16ms/step - loss: 0.1322 - accuracy: 0.9712
Epoch 10/10
25/25 [==============================] - 0s 17ms/step - loss: 0.1121 - accuracy: 0.9712
